In [8]:
import os
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from natsort import natsorted
import string

# Removing Heading and footer of the documents

In [14]:
def remove_header_footer(final_string):
    new_final_string = ""
    tokens = final_string.split('\n')
  
    # Remove tokens[0] and tokens[-1]
    for token in tokens[1:-1]:
        new_final_string += token+" "
        
    return new_final_string


# Reading the documents

In [15]:
def read_file(filename):
    with open(filename, 'r', encoding ="ascii", errors ="surrogateescape") as f:
        stuff = f.read()
  
    f.close()
      
    # Remove header and footer.
    stuff = remove_header_footer(stuff)
      
    return stuff

# Removing Punctuations and lower casing and setemming the words

In [16]:
def preprocessing(final_string):
        # Tokenize.
    tokenizer = TweetTokenizer()
    token_list = tokenizer.tokenize(final_string)
 
    # Remove punctuations.
    table = str.maketrans('', '', '\t')
    token_list = [word.translate(table) for word in token_list]
    punctuations = (string.punctuation).replace("'", "")
    trans_table = str.maketrans('', '', punctuations)
    stripped_words = [word.translate(trans_table) for word in token_list]
    token_list = [str for str in stripped_words if str]
 
    # Change to lowercase.
    token_list =[word.lower() for word in token_list]
    return token_list
 

# Creating the Positional Index

In [17]:
folder_names = ["files"]
 
# Initialize the stemmer.
stemmer = PorterStemmer()
 
# Initialize the file no.
fileno = 0
 
# Initialize the dictionary.
pos_index = {}
 
# Initialize the file mapping (fileno -> file name).
file_map = {}
 
for folder_name in folder_names:
 
    # Open files.
    file_names = natsorted(os.listdir("./" + folder_name))
 
    # For every file.
    for file_name in file_names:
 
        # Read file contents.
        stuff = read_file("./" + folder_name + "/" + file_name)
         
        # This is the list of words in order of the text.
        # We need to preserve the order because we require positions.
        # 'preprocessing' function does some basic punctuation removal,
        # stopword removal etc.

        final_token_list = preprocessing(stuff)
 
        # For position and term in the tokens.
        for pos, term in enumerate(final_token_list):
             
                    # First stem the term.
                    term = stemmer.stem(term)
 
                    # If term already exists in the positional index dictionary.
                    if term in pos_index:
                         
                        # Increment total freq by 1.
                        pos_index[term][0] = pos_index[term][0] + 1
                         
                        # Check if the term has existed in that DocID before.
                        if fileno in pos_index[term][1]:
                            pos_index[term][1][fileno].append(pos)
                             
                        else:
                            pos_index[term][1][fileno] = [pos]
 
                    # If term does not exist in the positional index dictionary
                    else:
                         
                        # Initialize the list.
                        pos_index[term] = []
                        # The total frequency is 1.
                        pos_index[term].append(1)
                        # The postings list is initially empty.
                        pos_index[term].append({})     
                        # Add doc ID to postings list.
                        pos_index[term][1][fileno] = [pos]
 
        # Map the file no. to the file name.
        file_map[fileno] = "./" + folder_name + "/" + file_name
 
        # Increment the file no. counter for document ID mapping             
        fileno += 1
pos_index

{'thi': [6, {0: [0], 1: [0], 2: [4, 13, 17], 3: [4]}],
 'is': [8, {0: [1], 1: [1], 2: [2, 5, 14, 18], 3: [2, 5]}],
 'the': [5, {0: [2], 1: [2], 2: [6, 15], 3: [6]}],
 'second': [4, {0: [3], 1: [3], 2: [7], 3: [7]}],
 'text': [4, {0: [4], 1: [4], 2: [8], 3: [8]}],
 'hello': [4, {0: [5], 1: [5], 2: [9], 3: [9]}],
 'how': [4, {0: [6], 1: [6], 2: [10], 3: [10]}],
 'are': [4, {0: [7], 1: [7], 2: [11], 3: [11]}],
 'you': [4, {0: [8], 1: [8], 2: [12], 3: [12]}],
 'my': [2, {2: [0], 3: [0]}],
 'name': [2, {2: [1], 3: [1]}],
 'shayanth': [2, {2: [3], 3: [3]}],
 'third': [1, {2: [16]}],
 'it': [1, {2: [19]}],
 'now': [1, {2: [20]}]}

In [18]:
test = pos_index["shayanth"]
print("################")
print("Positional Index")
print("frequency:"+str(test[0]))
print("################")
 
file_list = test[1]

for fileno, positions in file_list.items():
    print("Flie Location: "+str(file_map[fileno] )+"\n Position of query"+str( positions)+"\n################\n")

################
Positional Index
frequency:2
################
Flie Location: ./files/file4.txt
 Position of query[3]
################

Flie Location: ./files/file.txt
 Position of query[3]
################

